In [165]:
import pandas as pd
import numpy as np
from datetime import datetime
import statistics
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import Week
from pandas.tseries.offsets import BDay
from pandas.tseries.offsets import CustomBusinessDay

In [3]:
S5 = pd.read_csv('S5.csv')
S4 = pd.read_csv('S4.csv')
S6 = pd.read_csv('S6.csv')
con_s5 = pd.read_csv('SNP500.csv')
con_s4 = pd.read_csv('SNP400.csv')
con_s6 = pd.read_csv('SNP600.csv')

In [4]:
con_s5["index"] = "SNP500"
con_s4["index"] = "SNP400"
con_s6["index"] = "SNP600"

In [5]:
tic_null_s4 = con_s4[con_s4["thru"].isnull()]["co_tic"]
tic_null_s5 = con_s5[con_s5["thru"].isnull()]["co_tic"]
tic_null_s6 = con_s6[con_s6["thru"].isnull()]["co_tic"]

In [6]:
con_s5["thru"].replace(np.NaN, '31/01/2018', inplace=True)
con_s4["thru"].replace(np.NaN, '31/01/2018', inplace=True)
con_s6["thru"].replace(np.NaN, '31/01/2018', inplace=True)

In [8]:
S = pd.read_csv('cleaned.csv')

In [9]:
S = pd.concat([S6, S4, S5])
S["Index"] = "None"
indexs = pd.concat([con_s4, con_s5, con_s6])
tickers = pd.unique(indexs["co_tic"])
indexs["from"] = [datetime.strptime(x, '%d/%m/%Y') for x in indexs["from"]]
indexs["thru"] = [datetime.strptime(x, '%d/%m/%Y') for x in indexs["thru"]]

In [36]:
S_C = S[np.isnan(S["MC"]) != True] # & (S["Index"] != "None") ]
S_C = S_C.drop("Unnamed: 0", 1)
S_C["StdIndex"] = 0
S_C["AvgIndex"] = 0
S_C["IndChange"] = 0
S_C["Switch"] = 0
dates = pd.unique(S["date"])
dates = np.sort(dates)

In [ ]:
S_C.loc[(S_C["TICKER"] == "AMD") & (S_C["date"] <= "2017-01-01"), "Index"]

In [ ]:
x = 0
for i in tickers:
    x += 1
    print(x)
    dt = indexs[indexs["co_tic"] == i]
    sz = dt.shape[0]
    if  sz == 1:
        d = str(dt["thru"].iloc[0])[:10]
        ind = dt["index"].iloc[0]
        S_C.loc[(S_C["TICKER"] == i) & (S_C["date"] <= d), "Index"] = ind
        S_C.loc[(S_C["TICKER"] == i) & (S_C["date"] == d), "Switch"] = 1
    else:
        for j in range(0, sz):
            f = str(dt["from"].iloc[j])[:10]
            t = str(dt["thru"].iloc[j])[:10]
            ind = dt["index"].iloc[j]
            S_C.loc[(S_C["TICKER"] == i) & (S_C["date"] <= t)  & (S_C["date"] >= f), "Index"] = ind
            S_C.loc[(S_C["TICKER"] == i) & (S_C["date"] == f), "Switch"] = 1

S_C = S_C.drop_duplicates()
S_C.to_csv("cleand_w_index.csv")

In [168]:
S_C["StdIndex-1D"] = 0
S_C["StdIndex-1W"] = 0
S_C["StdIndex-4W"] = 0
S_C["MC-1D"] = 0
S_C["MC-1W"] = 0
S_C["MC-4W"] = 0
S_C["AvgIndex-1D"] = 0
S_C["AvgIndex-1W"] = 0
S_C["AvgIndex-4W"] = 0
S_C["R-0D"] = 0
S_C["R-1D"] = 0
S_C["R-1W"] = 0
S_C["R-4W"] = 0
change_tic = S_C[S_C["Switch"] == 1]["TICKER"]
change_tic = pd.unique(change_tic)

In [ ]:
k = 0
for tic in change_tic[24:]:
    k+=1
    print(k)
    print(tic)
    d_s = S_C[(S_C["Switch"] == 1) & (S_C["TICKER"] == tic)]["date"]
    StdIndex, StdIndex_1D, StdIndex_1W, StdIndex_4W , MC_1D, MC_1W, MC_4W, AvgIndex, AvgIndex_1D, \
    AvgIndex_1W ,AvgIndex_4W = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    for i in d_s:
        if i > min(S_C[S_C["TICKER"] == tic]["date"]):
            try:
                d_sc = i.split("-")
                d_minD1 = str(pd.datetime(int(d_sc[0]),int(d_sc[1]),int(d_sc[2])) - BDay(1))[:10]
                d_minW1 = str(pd.datetime(int(d_sc[0]),int(d_sc[1]),int(d_sc[2])) - Week(1))[:10]
                d_minW4 = str(pd.datetime(int(d_sc[0]),int(d_sc[1]),int(d_sc[2])) - Week(4))[:10]

                IndCur = S_C[(S_C["date"] == i) & (S_C["TICKER"] == tic)]["Index"].iloc[0]
                IndChange = S_C[(S_C["date"] == d_minD1) & (S_C["TICKER"] == tic)]["Index"]
                if IndChange.size == 0:
                    d_minD1 = str(pd.datetime(int(d_sc[0]),int(d_sc[1]),int(d_sc[2])) - BDay(2))[:10]
                    IndChange = S_C[(S_C["date"] == d_minD1) & (S_C["TICKER"] == tic)]["Index"]
                IndChange = IndChange.iloc[0]

                MC_1D = S_C[(S_C["date"] == d_minD1) & (S_C["Index"] == IndChange) & (S_C["TICKER"] == tic)]["MC"].iloc[0]
                MC_1W = S_C[(S_C["date"] == d_minW1) & (S_C["Index"] == IndChange) & (S_C["TICKER"] == tic)]["MC"]
                MC_4W = S_C[(S_C["date"] == d_minW4) & (S_C["Index"] == IndChange) & (S_C["TICKER"] == tic)]["MC"]

                if MC_1W.size == 0:
                    d_minW1 = str(pd.datetime(int(d_sc[0]),int(d_sc[1]),int(d_sc[2])) - Week(1) - BDay(1))[:10]
                    MC_1W = S_C[(S_C["date"] == d_minW1) & (S_C["Index"] == IndChange) & (S_C["TICKER"] == tic)]["MC"]
                MC_1W = MC_1W.iloc[0]

                if MC_4W.size == 0:
                    d_minW4 = str(pd.datetime(int(d_sc[0]),int(d_sc[1]),int(d_sc[2])) - Week(4) - BDay(1))[:10]
                    MC_4W = S_C[(S_C["date"] == d_minW4) & (S_C["Index"] == IndChange) & (S_C["TICKER"] == tic)]["MC"]
                MC_4W = MC_4W.iloc[0]
    
                StdIndex = S_C[(S_C["date"] == i) & (S_C["Index"] == IndChange)]["MC"].std()
                StdIndex_1D = S_C[(S_C["date"] == d_minD1) & (S_C["Index"] == IndChange)]["MC"].std()
                StdIndex_1W = S_C[(S_C["date"] == d_minW1) & (S_C["Index"] == IndChange)]["MC"].std()
                StdIndex_4W = S_C[(S_C["date"] == d_minW4) & (S_C["Index"] == IndChange)]["MC"].std()
                AvgIndex = S_C[(S_C["date"] == i) & (S_C["Index"] == IndChange)]["MC"].mean()
                AvgIndex_1D = S_C[(S_C["date"] == d_minD1) & (S_C["Index"] == IndChange)]["MC"].mean()
                AvgIndex_1W = S_C[(S_C["date"] == d_minW1) & (S_C["Index"] == IndChange)]["MC"].mean()
                AvgIndex_4W = S_C[(S_C["date"] == d_minW4) & (S_C["Index"] == IndChange)]["MC"].mean()

                new_df = pd.DataFrame({"AvgIndex":[AvgIndex], "StdIndex":[StdIndex_1D],"StdIndex-1W":[StdIndex_1W],
                                       "StdIndex-4W":[StdIndex_4W], "MC-1D":[MC_1D], "MC-1W":[MC_1W],"MC-4W":[MC_4W],
                                       "AvgIndex-1D": [AvgIndex_1D], "AvgIndex-1W":[AvgIndex_1W],
                                       "StdIndex-1D":[StdIndex_1D],"AvgIndex-4W":[AvgIndex_4W], "IndChange":[IndChange]},
                                      index = [S_C[(S_C["date"] == i) & (S_C["TICKER"] == tic)].index[0]])
                S_C.update(new_df)
            except:
                continue
        else:
            S_C.loc[(S_C["TICKER"] == tic) & (S_C["date"] == i), "Switch"] = 0
S_C = S_C.drop_duplicates()
S_C.to_csv("cleand_std_avg.csv")

In [337]:
S_C[(S_C["Switch"] == 1)].to_csv("changers.csv")

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
k = 0
for tic in change_tic[52:]:
    k+=1
    print(k)
    print(tic)
    d_s = S_C[(S_C["Switch"] == 1) & (S_C["TICKER"] == tic)]["date"]
    R_p1D, R_p0D, R_p1W, R_p4W = 0, 0, 0, 0
    for i in d_s:
        if i > min(S_C[S_C["TICKER"] == tic]["date"]):
            try:
                d_sc = i.split("-")
                d_mD1 = str(pd.datetime(int(d_sc[0]),int(d_sc[1]),int(d_sc[2])) - BDay(1))[:10]
                d_pD1 = str(pd.datetime(int(d_sc[0]),int(d_sc[1]),int(d_sc[2])) + BDay(1))[:10]
                d_pW1 = str(pd.datetime(int(d_sc[0]),int(d_sc[1]),int(d_sc[2])) + Week(1))[:10]
                d_pW4 = str(pd.datetime(int(d_sc[0]),int(d_sc[1]),int(d_sc[2])) + Week(4))[:10]
                
                P_m1D = S_C[(S_C["date"] == d_mD1) & (S_C["TICKER"] == tic)]["PRC"].iloc[0]
                P_p0D = S_C[(S_C["date"] == i)     & (S_C["TICKER"] == tic)]["PRC"].iloc[0]
                P_p1D = S_C[(S_C["date"] == d_pD1) & (S_C["TICKER"] == tic)]["PRC"].iloc[0]
                P_p1W = S_C[(S_C["date"] == d_pW1) & (S_C["TICKER"] == tic)]["PRC"].iloc[0]
                P_p4W = S_C[(S_C["date"] == d_pW4) & (S_C["TICKER"] == tic)]["PRC"].iloc[0]
                
                R_p0D = (P_p0D - P_m1D)/P_m1D
                R_p1D = (P_p1D - P_m1D)/P_m1D
                R_p1W = (P_p1W - P_m1D)/P_m1D
                R_p4W = (P_p4W - P_m1D)/P_m1D
                
                new_df = pd.DataFrame({"R-0D":[R_p0D], "R-1D":[R_p1D],"R-1W":[R_p1W], "R-4W":[R_p4W]},
                                      index = [S_C[(S_C["date"] == i) & (S_C["TICKER"] == tic)].index[0]])
                S_C.update(new_df)
            except:
                continue


In [386]:
D = S_C[ (S_C["Switch"] == 1) & (S_C["IndChange"] != 0) & (S_C["R-0D"] != 0) ]
D["toINDfrom"] = D["Index"] + D["IndChange"]
D.to_csv("temp_date.csv")

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
D = S_C[ (S_C["Switch"] == 1) & (S_C["IndChange"] != 0)]
D["toINDfrom"] = D["Index"] + D["IndChange"]
D.to_csv("data_thesis.csv")

In [321]:
S_C[ (S_C["Switch"] == 1) & (S_C["IndChange"] != 0)].to_csv("data_thesis.csv")